# Tutorial : Spectral early warning signals with *ewstools*

By the end of this tutorial you should know how to:
- Compute and visualise the power spectrum over a rolling window
- Compute spectral early warning signals as in [Bury et al. (2021) Royal Soc. Interface](https://royalsocietypublishing.org/doi/full/10.1098/rsif.2020.0482)

Total run time : X min Y s on Macbook Air (M1, 2020)


## Import libraries

In [1]:
# Start timer to record execution time of notebook
import time
start_time = time.time()

In [2]:
import numpy as np
np.random.seed(0) # Set seed for reproducibility
import pandas as pd
import matplotlib.pyplot as plt
import os

import ewstools
from ewstools.models import simulate_ricker

## Simulate model data

The power spectrum of a time series evolves in different ways preceding a Hopf vs. a fold bifurcation (see e.g. [Wiesenfeld (1985)](https://link.springer.com/article/10.1007/BF01010430)). Therefore EWS derived from the power spectrum have the potential to discriminate between these two types of bifurcation. Let's demonstrate this with model simulations. For a fold bifurcation, we will use the Ricker model with a nonlinear harvesting term. For the Hopf bifurcation, we will use the Rosenzweig–MacArthur consumer-resource model.